### 这一节我们主要学习
+ 为什么使用VAE
+ 高斯混合模型（Gaussian Mixture Model）
+ VAE的问题
+ 生成对抗网络（GAN）

### 1. 为什么使用VAE？




![](http://imgbed.momodel.cn/29_01_ul_VAE.png)

为什么要用VAE？<br>
一个直觉的解释是， auto-encoder 的 encoder 与 decoder 都是非线性的，<br>
使用 auto-encoder 时，中间的 code 与满月的 code 和弦月的 code 都有偏差，所以很难预测问号处是一张介于满月与弦月之间的图像。<br>
而 VAE 使得 code 加上噪声之后还能恢复出来，<br>
这样问号处的 code 虽然与满月或弦月的 code 有偏差可仍有可能恢复出介于两者之间的图像。<br>


![](http://imgbed.momodel.cn/29_02_ul_VAE.png)

VAE 是在 encoder 的输出上加噪声，<br>
而 De-noising auto-encoder 是在 encoder input 上加噪声 。 <br>
如果只最小化重构误差，那么训练出来的结果不会是预期的样子，<br>
因为方差是学出来的，会学成 0。 <br>


![](http://imgbed.momodel.cn/29_03_ul_VAE.png)

exp 是为了确保方差大小是正数，<br>
右下方的 minimize 是为了让方差不能太小，<br>
其中第一项为蓝色曲线、第二项为红色曲线、二者相减得绿色曲线，这样方差最小为1，<br>
第三项是 L2 正则化，希望参数稀疏避免过拟合。<br>

![](http://imgbed.momodel.cn/29_04_ul_VAE.png)

上图中用一维表示的是20X20高维空间，然后估计其概率分布P(x)；<br>
然后根据概率分布生成一张宝可梦的图片；<br>
如果图片是宝可梦的的话，其几率比较高，不像宝可梦时其几率比较低。

### 2. 高斯混合模型（Gaussian Mixture Model）





![](http://imgbed.momodel.cn/29_05_ul_VAE.png)

通常对于 VAE 的解释如下，每一个图片可以看作是高维空间的一个点，<br>
而 VAE 则是估计高维空间的点分布，即计算出 P(x)，选取的输出在概率较大的地方。<br>
这里使用高斯混合模型进行解释，如上图所示：<br>
复杂的概率分布可以看作是多个高斯分布通过不同的加权叠加的结果。<br>
在进行概率分布计算时，首先决定从哪一个高斯分布进行取样，<br>
然后再计算该高斯分布的概率，这也就对应的全概率公式的形式。<br> 
首先，从 multinomial distribution（多项式分布）中选取m，此时m为正整数；<br>
对于每一个m，都存在均值和方差，则可以得到m对应的高斯分布，最后得到p(x|m)。<br>
<br>
对于实际操作，需要决定有多少个 mixture ，<br>
然后根据数据，采用 EM 方法，可以得到权重、均值和方差。<br>
进一步地，我们可以采用离散分布估计来进行 VAE 操作，而不是局限于此处的聚类操作。<br>


![](http://imgbed.momodel.cn/29_06_ul_VAE.png)



VAE 是高斯混合模型的distributive representation 的版本。<br>
首先，从正态分布选取一个z，z为向量，每一个维度代表要选取的每一个特征；<br>
然后根据z得到高斯分布的 $μ,σ$ 假设为 $μ(z),σ(z)$ ，也就是前述的变量都是关于z的一个分布，<br>
这个分布同样可以使用神经网络表示，输入为z，输出为均值向量，<br>
对于方差矩阵，可以将其向量化或者采用输出对角线元素， non-diagonal 元素为0，<br>
最终的概率为$P(x)$。



![](http://imgbed.momodel.cn/29_07_ul_VAE.png)

对于 code space 中的一个向量z，向量z符合某个分布（一般为高斯分布），<br>
需要利用神经网络表示出$μ(z),σ(z)$，<br>
输出的结果在进行完全的 decode 后，得到x，需要保证上述p(x)的似然函数最大。<br>

![](http://imgbed.momodel.cn/29_09_ul_VAE.png)

将似然函数 P(x) 分解，其中 q(z|x) 可以为任意概率分布，
最后化简的右侧项为 KL-divergence ，衡量两个概率分布的相似程度（相对熵），
得到了下边界$L_b$。 

![](http://imgbed.momodel.cn/29_10_ul_VAE.png)

$logP(x)$只与P(x|z)有关，与q(z|x)无关；<br>
寻找P(x|z)和q(z|x),使得$L_b$最大；<br>
即：找到合适的q(z|x)使得$L_b$与$logP(x)$越来越接近；<br>
而如果能够最大化$L_b$，则会最小化KL，<br>
这意味着似然函数 logP(x) 与$L_b$逐渐靠近。<br>
最后使得：q(z|x)越来越接近p(z|x)


![](http://imgbed.momodel.cn/29_11_ul_VAE.png)

最后化简$L_b$式子如上图，包含了一个 KL-divergence 和一个积分项，<br>
需要最小化 KL-divergence ，对应于 regularization 项的前两个。<br>

![](http://imgbed.momodel.cn/29_12_ul_VAE.png)

最小化 KL(q(z|x)||P(z)) ：<br>
对于x，进入 encoder 神经网络，表示出$μ^{'}(x),σ^{'}(x)$，<br>
其实以上过程就等价于上图中黄色方框那部分内容。具体可以参考后面的论文<br>
<br>
最大化积分项：<br>
首先对于x，进入 encoder ，得到x的 mean 和 variance ；<br>
其次根据x的 mean 和 variance 得到z；<br>
然后将z输入到 decoder 神经网络，使得到的$μ(x)$越接近x越好。



### 3. VAE的问题






![](http://imgbed.momodel.cn/29_13_ul_VAE.png)

VAE 的问题是其实 VAE 并没有想过从数据中学习怎样去生成一张新图，<br>
而是在产生一张与数据中的图尽可能相似的图，只是在模仿。<br>

上图中一个像素点的不同在原始的“7”图上，左边的可以理解，<br>
但是右图是不一样的，是假的，但是VAE会认为说其与原始目标是一致的。<br>
**VAE 并没有生成新图，而是仅仅记住了已经存在了的图。**<br>


### 4. 生成对抗网络（GAN）






Yann LeCun 说过， GAN 是 ML 领域近十年最有意思的想法。

#### 4.1 介绍GAN概念
<br>
用拟态的演化来类比 GAN ，下面介绍枯叶蝶的演化过程：

![](http://imgbed.momodel.cn/29_14_ul_GAN.png)

假设枯叶蝶最初也是彩色的，但是其有天敌，类似于麻雀等；<br>
假如说蝴蝶的天敌认为蝴蝶不是棕色的，所以蝴蝶就演化为棕色的；<br>
与此同时，蝴蝶的天敌也会进行演化，此时认为蝴蝶没有叶脉；<br>
因此蝴蝶需要继续演化，演化为枯叶蝶。而此时其天敌也会继续演化...

![](http://imgbed.momodel.cn/29_15_ul_GAN.png)

Discriminator v1 判断 image 是 Generator v1 产生的还是 real images,然后据此更新参数形成 Generator v2；<br>
Discriminator v2 判断 image 是 Generator v2 产生的还是 real images,然后据此更新参数形成 Generator v3；<br>
Discriminator v3 判断 image 是 Generator v3 产生的还是 real images,然后据此更新参数形成新的Generator；<br>
Generator v3产生的image更像真实的image，它可以骗过Discriminator v2；Generator v2产生的image可以骗过Discriminator v1；<br>
与此同时，判别网络Discriminator也在演化。
因此,Generator根据Discriminator演化。 <br>
**Generator从未看过real images, 产生的是database中没有见过的image**。

#### 4.2 训练

+ 判别网络（Discriminator）

![](http://imgbed.momodel.cn/29_16_ul_GAN.png)

判别网络模型的输入是image(可以是真实图片，也可以是生成网络生成的图片)<br>
判别网络模型的输出是标量，可以通过 sigmoid 使输出值介于0-1之间；<br>
可以通过设置阈值表示期望结果的可信度，例如希望判断为真实图片的结果尽可能正确，可以设置当输出大于0.8时，标注为1。<br>
输出为1表示输入的图片是真实的，否则输入的图像是假的,是生成模型生成的。<br>
<br>
生成网络的架构与 VAE 的 decoder 部分相似；也是一个神经网络模型。<br>
输入是一个随机分布的向量，输入到生成网络中得到一个 image ；<br>
输入不同的向量，得到不同的 image ；<br>
将输出的 image 都标注为0，即 fake ；<br>
将真实图片都标注为1，即 real ；<br>

+ 生成网络（Generator）

![](http://imgbed.momodel.cn/29_17_ul_GAN.png)

随机输入一个向量到生成网络模型中，会得到一张图片；<br>
然后将图片输入到判别网络模型中，得到一个标量，比如0.87；<br>
然后开始调整生成网络模型的参数，使其生成的图片越真实越好；<br>
即将其生成的图片放入判别网络模型中，使其输出越接近1越好。<br>
在以上的过程中需要固定住判别网络模型的参数。

+ GAN - Toy Example

![](http://imgbed.momodel.cn/29_18_ul_GAN.png)

图中（黑点： real ，绿线： Generator ，蓝线： Discriminator ）

将Z输入到生成网络模型中生成x(上图中绿色曲线)，<br>
然后将x输入到判别网络中去，得到结果(图中蓝色曲线)；<br>
然后通过蓝色曲线了解生成x与真实图片(黑色曲线)之间的区别；<br>
直至蓝色曲线成为一条直线，表示生成图片x与真实图片一模一样，判别网络识别不出来。<br>

#### 4.3  GAN 遇到的问题

![](http://imgbed.momodel.cn/29_19_ul_GAN.png)

#### 4.4 参考资料

+ “Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks”
+ “Improved Techniques for Training GANs”
+ “Autoencoding beyond pixels using a learned similarity metric”
+ “Deep Generative Image Models using a Laplacian Pyramid of Adversarial Network”
+ “Super Resolution using GANs”
+ “Generative Adversarial Text to Image Synthesis”

+ 基本材料：
    + http://blog.aylien.com/introduction-generative-adversarial-networks-code-tensorflow/
    + https://bamos.github.io/2016/08/09/deep-completion/
    + http://blog.evjang.com/2016/06/generative-adversarial-nets-in.html

### 5. 总结
+ VAE 的原理及其优缺点
+ GAN 的原理